In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
from scripts.gestion_donnees.pretraitement import Pretraitement
from scripts.evaluation.evaluation_model import Evaluation
from scripts.classifieurs.forets_aleatoires import Forets_aleatoires
from scripts.classifieurs.reseaux_de_neurones import Reseaux_de_neurones
from scripts.classifieurs.arbre_de_decision import Arbre_de_decision
from scripts.classifieurs.svm import SVM_Classificateur
from scripts.classifieurs.adaboost import AdaBoost_model
from scripts.classifieurs.bagging import Bagging
from scripts.classifieurs.knn import Knn 
from scripts.classifieurs.logistic_regression import LogisticRegression_model
from scripts.classifieurs.perceptron import Perceptron_model
from scripts.classifieurs.comninaison import Comninaison
from sklearn.ensemble import RandomForestClassifier

## Partie 1 :  Prétraitement et analyse des données

In [2]:
# creation des instances
Ev = Evaluation
Pt = Pretraitement("./data/train.csv")
#  charger donneés entrainement
df_train_init = Pt.Charger_donnees()
# encoder donneés
etiquette,classes = Pt.Encoder_donnees(df_train_init,"species")
df_train_init = df_train_init.drop(['id','species'], axis=1)
# supprimer les valeurs aberrantes
outliers = Pt.Indice_outliers(df_train_init,3.0)
df_train = df_train_init.drop(outliers)
etiquette = [etiquette[i] for i in range(len(etiquette)) if i not in outliers]
# diviser les donnees
x_train, x_test, y_train, y_test, x_val, y_val = Pt.Diviser_donnees(df_train,etiquette)


In [ ]:
#vérifie la redondance des datas
print("Le nombre de données redondantes de train est " + str(df_train.duplicated().sum()))
#vérifie si pour un des attributs il manque une valeur
print("Nb valeur manquante train " + str(df_train.isnull().any().sum()))
#
unique_elements = set(etiquette)

# Get the count of unique elements
n_classes = len(unique_elements)
print(n_classes)


## Partie 2 : Modélisation et évaluation des données

##### Forets aléatoires

In [ ]:
model_foret_alea = Forets_aleatoires(x_train, y_train, x_val, y_val,x_test,y_test)
model_foret_alea.validation_croisee_gridsearch()
model_foret_alea.entrainement()
predictions = model_foret_alea.prediction()

In [ ]:
evaluation_fa= Evaluation(model=model_foret_alea.rf_classifier, x_test=x_test, y_test=y_test, x_train=x_train, y_train=y_train)
f1, precision, recall = evaluation_fa.calculate_metrics(model_foret_alea.y_test, model_foret_alea.prediction())
print(f"F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
evaluation_fa.plt_roc_curve()
evaluation_fa.plt_confusion_matrix()
evaluation_fa.generate_classification_report()
train_sizes = np.linspace(0.1, 1.0, 10)
evaluation_fa.plot_learning_curves(train_sizes=train_sizes, scoring='accuracy')
evaluation_fa.plot_learning_curves_loss(train_sizes=train_sizes, scoring='neg_log_loss')

##### SVM

In [ ]:
svm_model = SVM_Classificateur(x_train, y_train, x_val, y_val, x_test, y_test)
svm_model.validation_croisee_gridsearch()
svm_model.entrainement()
predictions_svm = svm_model.prediction()

In [ ]:
evaluation_svm = Evaluation(svm_model.svm_classifier, x_train, y_train, x_test, y_test)
f1, precision, recall = evaluation_svm.calculate_metrics(evaluation_svm.y_test, predictions_svm)
print(f"F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
# evaluation_svm.plt_roc_curve()
# evaluation_svm.plt_confusion_matrix()
# evaluation_svm.generate_classification_report()
train_sizes = np.linspace(0.1, 1.0, 10)
evaluation_svm.plot_learning_curves(train_sizes=train_sizes, scoring='accuracy')
#evaluation_svm.plot_learning_curves_loss(train_sizes=train_sizes, scoring='neg_log_loss')

##### Arbre de decision

In [3]:
Arbre_de_decision_model = Arbre_de_decision(x_train, y_train, x_val, y_val, x_test, y_test)
Arbre_de_decision_model.validation_croisee_gridsearch()
Arbre_de_decision_model.entrainement()
predictions = Arbre_de_decision_model.prediction()

c:\Users\mazen\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
exception calling callback for <Future at 0x19c98fb5c70 state=finished returned list>
Traceback (most recent call last):
  File "c:\Users\mazen\AppData\Local\Programs\Python\Python39\lib\site-packages\joblib\externals\loky\_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "c:\Users\mazen\AppData\Local\Programs\Python\Python39\lib\site-packages\joblib\parallel.py", line 359, in __call__
    self.parallel.dispatch_next()
  File "c:\Users\mazen\AppData\Local\Programs\Python\Python39\lib\site-packages\joblib\parallel.py", line 792, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "c:\Users\mazen\AppData\Local\Programs\Python\Python39\lib\site-packages\joblib\parallel.py", l

KeyboardInterrupt: 

In [ ]:
evaluation_dt = Evaluation(Arbre_de_decision_model.dt_classifier, Arbre_de_decision_model.x_test, Arbre_de_decision_model.y_test,Arbre_de_decision_model.x_train,Arbre_de_decision_model.y_train)
f1, precision, recall = evaluation_dt.calculate_metrics(Arbre_de_decision_model.y_test, Arbre_de_decision_model.prediction())
print(f"F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
# evaluation_dt.plt_roc_curve()
# evaluation_dt.plt_confusion_matrix()
# evaluation_dt.generate_classification_report()
train_sizes = np.linspace(0.1, 1.0, 10)
evaluation_dt.plot_learning_curves(train_sizes=train_sizes, scoring='accuracy')


##### Ada boosting


In [ ]:
adaboost_model = AdaBoost_model(x_train, y_train, x_val, y_val, x_test, y_test)
adaboost_model.validation_croisee_gridsearch()
adaboost_model.entrainement()
predictions = adaboost_model.prediction()

In [ ]:
evaluation_ab = Evaluation(adaboost_model.ab_classifier, adaboost_model.x_test, adaboost_model.y_test,adaboost_model.x_train,adaboost_model.y_train)
f1, precision, recall = evaluation_ab.calculate_metrics(adaboost_model.y_test, adaboost_model.prediction())
# print(f"F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
# evaluation_ab.plt_roc_curve()
# evaluation_ab.plt_confusion_matrix()
# evaluation_ab.generate_classification_report()
train_sizes = np.linspace(0.1, 1.0, 10)
evaluation_ab.plot_learning_curves(train_sizes=train_sizes, scoring='accuracy')

##### Reseaux de neurones


In [ ]:
model_R_N = Reseaux_de_neurones(x_train, y_train, x_val, y_val, x_test, y_test)
model_R_N.preprocess_data()
model_R_N.garder_meilleur_hyperparameters()
model_R_N.entrainement()
model_R_N.prediction()

In [ ]:
evaluation_rn = Evaluation(model_R_N.nn_classifier, model_R_N.x_test, model_R_N.y_test,model_R_N.x_train,model_R_N.y_train)
f1, precision, recall = evaluation_rn.calculate_metrics(model_R_N.y_test, model_R_N.prediction())
print(f"F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
# evaluation_rn.plt_roc_curve()
# evaluation_rn.plt_confusion_matrix()
# evaluation_rn.generate_classification_report()
train_sizes = np.linspace(0.1, 1.0, 10)
evaluation_rn.plot_learning_curves(train_sizes=train_sizes, scoring='accuracy')

In [ ]:
print("Best hyperparameters:", model_R_N.best_hyperparameters)

##### Bagging

In [ ]:
bagging_classifier = Bagging(x_train, y_train, x_val, y_val, x_test, y_test)
bagging_classifier.garder_meilleur_hyperparameters()
bagging_classifier.entrainement()
predictions = bagging_classifier.prediction()

In [ ]:
evaluation_bg = Evaluation(model=bagging_classifier.bc_classifier, x_test=x_test, y_test=y_test, x_train=x_train, y_train=y_train)
f1, precision, recall = evaluation_bg.calculate_metrics(bagging_classifier.y_test, bagging_classifier.prediction())
print(f"F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
# evaluation_bg.plt_roc_curve()
# evaluation_bg.plt_confusion_matrix
# evaluation_bg.generate_classification_report()
train_sizes = np.linspace(0.1, 1.0, 10)
evaluation_bg.plot_learning_curves(train_sizes=train_sizes, scoring='accuracy')


##### KNN

In [ ]:
KNN_classifier = Knn(x_train, y_train, x_val, y_val, x_test, y_test)
KNN_classifier.validation_croisee_gridsearch()
KNN_classifier.entrainement()
predictions = KNN_classifier.prediction()

In [ ]:
evaluation_knn = Evaluation(KNN_classifier.knn_classifier, x_train, y_train, x_test, y_test)
f1, precision, recall = evaluation_knn.calculate_metrics(KNN_classifier.y_test, KNN_classifier.prediction())
print(f"F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
# evaluation_knn.plt_roc_curve()
# evaluation_knn.plt_confusion_matrix()
# evaluation_knn.generate_classification_report()
train_sizes = np.linspace(0.1, 1.0, 10)
evaluation_knn.plot_learning_curves(train_sizes=train_sizes,scoring='accuracy')
evaluation_knn.plot_learning_curves_loss(train_sizes=train_sizes)
evaluation_knn.plot_learning_curves_accuracy(train_sizes=train_sizes)


##### Perceptron

In [ ]:
Perceptron_classifier = Perceptron_model(x_train, y_train, x_val, y_val, x_test, y_test)
Perceptron_classifier.validation_croisee_gridsearch()
Perceptron_classifier.entrainement()
predictions = Perceptron_classifier.prediction()

In [ ]:
evaluation_P = Evaluation(Perceptron_classifier.perceptron_classifier, x_train, y_train, x_test, y_test)
f1, precision, recall = evaluation_P.calculate_metrics(Perceptron_classifier.y_test, Perceptron_classifier.prediction())
print(f"F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
# evaluation_P.plt_roc_curve()
# evaluation_P.plt_confusion_matrix()
# evaluation_P.generate_classification_report()
train_sizes = np.linspace(0.1, 1.0, 10)
evaluation_P.plot_learning_curves(train_sizes=train_sizes, scoring='accuracy')
#evaluation_P.plot_learning_curves_loss(train_sizes=train_sizes,scoring='neg_mean_squared_error')


##### Logistic regression

In [ ]:
RL_classifier = LogisticRegression_model(x_train, y_train, x_val, y_val, x_test, y_test)
RL_classifier.validation_croisee_gridsearch()
RL_classifier.entrainement()
predictions = RL_classifier.prediction()

In [ ]:
evaluation_RL = Evaluation(RL_classifier.logistic_regression_classifier, x_train, y_train, x_test, y_test)
f1, precision, recall = evaluation_RL.calculate_metrics(RL_classifier.y_test, RL_classifier.prediction())
print(f"F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
# evaluation_RL.plt_roc_curve()
# evaluation_RL.plt_confusion_matrix()
# evaluation_RL.generate_classification_report()
train_sizes = np.linspace(0.1, 1.0, 10)
evaluation_RL.plot_learning_curves(train_sizes=train_sizes, scoring='accuracy')

##### Combinaison

In [ ]:
GB_classifier = Comninaison(x_train, y_train, x_val, y_val, x_test, y_test)
GB_classifier.validation_croisee_gridsearch()
GB_classifier.entrainement()
predictions = GB_classifier.prediction()

In [ ]:
evaluation_GB = Evaluation(GB_classifier.ensemble_classifier, x_train, y_train, x_test, y_test)
f1, precision, recall = evaluation_GB.calculate_metrics(GB_classifier.y_test, GB_classifier.prediction())
print(f"F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
# evaluation_RL.plt_roc_curve()
# evaluation_RL.plt_confusion_matrix()
# evaluation_RL.generate_classification_report()
train_sizes = np.linspace(0.1, 1.0, 10)
evaluation_GB.plot_learning_curves(train_sizes=train_sizes, scoring='accuracy')